In [ ]:
!pip install anomalib[full]

In [1]:
from typing import Any
import time
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from torchvision.transforms import ToPILImage
import pandas as pd

from anomalib.data import MVTecAD
from anomalib.data.utils import read_image
from anomalib.deploy import ExportType, OpenVINOInferencer
from anomalib.engine import Engine
from anomalib.models import EfficientAd
from anomalib.utils.visualization import ImageResult


/usr/local/lib/python3.11/dist-packages/openvino/runtime/__init__.py:10: DeprecationWarning: The `openvino.runtime` module is deprecated and will be removed in the 2026.0 release. Please replace `openvino.runtime` with `openvino`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [ ]:
import os
import torch
import time
import contextlib
from anomalib.data import MVTecAD
from anomalib.models import EfficientAd
from pytorch_lightning import Trainer
from statistics import mean

# Categories from MVTec AD
categories = [
    'bottle', 'cable', 'capsule', 'carpet', 'grid',
    'hazelnut', 'leather', 'metal_nut', 'pill', 'screw',
    'tile', 'toothbrush', 'transistor', 'wood', 'zipper'
]

# To store results
all_results = []

# Helper to suppress output
@contextlib.contextmanager
def suppress_output():
    with open(os.devnull, "w") as devnull:
        with contextlib.redirect_stdout(devnull), contextlib.redirect_stderr(devnull):
            yield

with suppress_output():

    # Training loop
    for category in categories:
        print(f"Processing category: {category}")
        start_time = time.time()

        datamodule = MVTecAD(
            root='./datasets/MVTecAD',
            category=category,
            train_batch_size=1,
            eval_batch_size=1,
            num_workers=2,
        )

        datamodule.prepare_data()
        datamodule.setup()

        model = EfficientAd()

        engine = Engine(
            max_epochs = 10,
            accelerator="auto",  # \<"cpu", "gpu", "tpu", "ipu", "hpu", "auto">,
            devices=1,
            logger=False,
        )

        engine.fit(model, datamodule=datamodule)
        result = engine.test(model, datamodule=datamodule)

        result = result[0]
        latency = time.time() - start_time
        result["category"] = category
        result["latency_sec"] = round(latency, 2)

        all_results.append(result)

# Compute averages
avg_metrics = {
    "image_AUROC": mean([r["image_AUROC"] for r in all_results]),
    "image_F1Score": mean([r["image_F1Score"] for r in all_results]),
    "latency_sec": mean([r["latency_sec"] for r in all_results])
}

# Display summary
print("\nPer-Category Results:")
for r in all_results:
    print(f"{r['category']:12s} | "
          f"Image AUROC: {r['image_AUROC']:.4f} | "
          f"Image F1: {r['image_F1Score']:.4f} | "
          f"Latency: {r['latency_sec']}s")

print("\nAverage Metrics:")
for k, v in avg_metrics.items():
    print(f"{k}: {v:.4f}")


In [ ]:
# Display summary
print("\nPer-Category Results:")
for r in all_results:
    print(f"{r['category']:12s} | "
          f"Image AUROC: {r['image_AUROC']:.4f} | "
          f"Image F1: {r['image_F1Score']:.4f} | "
          f"Pixel AUROC: {r['pixel_AUROC']:.4f} | "
          f"Pixel F1: {r['pixel_F1Score']:.4f} | "
          f"Latency: {r['latency_sec']}s")

print("\nAverage Metrics:")
for k, v in avg_metrics.items():
    print(f"{k}: {v:.4f}")


Per-Category Results:
bottle       | Image AUROC: 1.0000 | Image F1: 0.9920 | Pixel AUROC: 0.9676 | Pixel F1: 0.7416 | Latency: 865.41s
cable        | Image AUROC: 0.9108 | Image F1: 0.8804 | Pixel AUROC: 0.9504 | Pixel F1: 0.5881 | Latency: 597.44s
capsule      | Image AUROC: 0.5975 | Image F1: 0.9030 | Pixel AUROC: 0.9083 | Pixel F1: 0.2462 | Latency: 558.43s
carpet       | Image AUROC: 0.9928 | Image F1: 0.9773 | Pixel AUROC: 0.9589 | Pixel F1: 0.6813 | Latency: 630.7s
grid         | Image AUROC: 0.9841 | Image F1: 0.9821 | Pixel AUROC: 0.9229 | Pixel F1: 0.4849 | Latency: 506.09s
hazelnut     | Image AUROC: 0.8296 | Image F1: 0.8400 | Pixel AUROC: 0.7892 | Pixel F1: 0.4989 | Latency: 802.34s
leather      | Image AUROC: 0.9178 | Image F1: 0.9130 | Pixel AUROC: 0.9588 | Pixel F1: 0.5503 | Latency: 569.69s
metal_nut    | Image AUROC: 0.9541 | Image F1: 0.9418 | Pixel AUROC: 0.9409 | Pixel F1: 0.7638 | Latency: 462.25s
pill         | Image AUROC: 0.9015 | Image F1: 0.9298 | Pixel AURO

##EfficientAD S

In [ ]:
MVTec = MVTecAD(
    root = './datasets/MVTecAD',
    category='bottle',
    train_batch_size=1,
    eval_batch_size=1,
    num_workers=0)

MVTec.prepare_data()
MVTec.setup()

model = EfficientAd()

engine = Engine(
    max_epochs = 10,
    accelerator="auto",  # \<"cpu", "gpu", "tpu", "ipu", "hpu", "auto">,
    devices=1,
    logger=False,
)

# Training
start_time = time.time()
engine.fit(model=model, datamodule=MVTec)
training_time = time.time() - start_time

# Testing
start_time = time.time()
test_result = engine.test(model=model, datamodule=MVTec)
inference_time = time.time() - start_time


In [12]:
print(test_result)

[{'image_AUROC': 1.0, 'image_F1Score': 0.9919999837875366, 'pixel_AUROC': 0.9697375893592834, 'pixel_F1Score': 0.7515456080436707}]


In [13]:
print(f"Training time: {training_time} seconds")
print(f"Inference time: {inference_time} seconds")

Training time: 542.5230667591095 seconds
Inference time: 15.021449565887451 seconds


In [14]:
print(f"Latency: {inference_time / len(MVTec.test_dataloader().dataset)}")

Latency: 0.18098132007093315


##EfficientAD M

In [ ]:
MVTec = MVTecAD(
    root = './datasets/MVTecAD',
    category='bottle',
    train_batch_size=1,
    eval_batch_size=1,
    num_workers=0)

MVTec.prepare_data()
MVTec.setup()

model = EfficientAd(model_size = "medium")

engine = Engine(
    max_epochs = 10,
    accelerator="auto",  # \<"cpu", "gpu", "tpu", "ipu", "hpu", "auto">,
    devices=1,
    logger=False,
)

# Training
start_time = time.time()
engine.fit(model=model, datamodule=MVTec)
training_time = time.time() - start_time

# Testing
start_time = time.time()
test_result = engine.test(model=model, datamodule=MVTec)
inference_time = time.time() - start_time

In [4]:
print(test_result)

[{'image_AUROC': 0.990476131439209, 'image_F1Score': 0.9599999785423279, 'pixel_AUROC': 0.9526698589324951, 'pixel_F1Score': 0.681119441986084}]


In [5]:
print(f"Training time: {training_time} seconds")
print(f"Inference time: {inference_time} seconds")

Training time: 1059.2677891254425 seconds
Inference time: 19.093512773513794 seconds


##FastFlow

In [2]:
from anomalib.models import Fastflow
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
datamodule = MVTecAD(
    root='./datasets/MVTecAD',
    category="bottle",
    train_batch_size=32,
    eval_batch_size=32,
    num_workers=0,
)

datamodule.prepare_data()
datamodule.setup()

model = Fastflow(backbone="resnet18", flow_steps=8)

callbacks = [
    ModelCheckpoint(
        mode="max",
        monitor="pixel_AUROC",
    ),
    EarlyStopping(
        monitor="pixel_AUROC",
        mode="max",
        patience=3,
    ),
]

engine = Engine(
    callbacks=callbacks,
    accelerator="auto",  # \<"cpu", "gpu", "tpu", "ipu", "hpu", "auto">,
    devices=1,
    logger=False,
)

In [ ]:
start_time = time.time()
engine.fit(model=model, datamodule=datamodule)
training_time = time.time() - start_time

In [ ]:
# Testing
start_time = time.time()
test_results = engine.test(model=model, datamodule=datamodule)
inference_time = time.time() - start_time

In [7]:
test_results

[{'image_AUROC': 0.9992063641548157,
  'image_F1Score': 0.9841269850730896,
  'pixel_AUROC': 0.9734765291213989,
  'pixel_F1Score': 0.6583619713783264}]

In [8]:
print(test_results)

[{'image_AUROC': 0.9992063641548157, 'image_F1Score': 0.9841269850730896, 'pixel_AUROC': 0.9734765291213989, 'pixel_F1Score': 0.6583619713783264}]


In [9]:
print(f"Training time: {training_time} seconds")
print(f"Inference time: {inference_time} seconds")

Training time: 223.50394463539124 seconds
Inference time: 12.50313663482666 seconds


In [10]:
print(f"Latency: {inference_time / len(datamodule.test_dataloader().dataset)}")

Latency: 0.1506402004195983
